In [59]:
from bs4 import BeautifulSoup
import time
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
import pandas as pd

In [60]:
def get_game(url):
    bets = []
    service = Service(executable_path="chromedriver.exe")
    driver = webdriver.Chrome(service=service)
    driver.get(url)
    html_content = driver.page_source
    soup = BeautifulSoup(html_content, 'html.parser')
    game_title = soup.find('span', class_='o_match-title').text.strip()
    time.sleep(8)  
   
    accordion_button = driver.find_element(By.XPATH, "//button[@class='o_accordion-item__trigger']//span[contains(text(), 'Genaues Ergebnis')]")
    accordion_button.click()

    time.sleep(2)  

    accordion_item = accordion_button.find_element(By.XPATH, "./ancestor::div[contains(@class, 'o_accordion-item')]")

    if 'o_accordion-item--expanded' in accordion_item.get_attribute('class'):
        panel = accordion_item.find_element(By.CLASS_NAME, 'o_accordion-item__panel')
        panel_html_content = panel.get_attribute('innerHTML')
        panel_soup = BeautifulSoup(panel_html_content, 'html.parser')
        bet_items_section = panel_soup.find_all('div', class_='o_bet-item')

        for section in bet_items_section:
            header = section.find('div', class_='o_bet-item__header').text.strip()
            if "Genaues Resultat (reg.Spielzeit)" in header:
                bet_items = section.find('div', class_='o_bet-item__content ng-star-inserted').find_all('button', class_='m_bet-button')
                for item in bet_items:
                    name = item.find('span', class_='m_bet-button__name').text.strip()
                    odds = item.find('span', class_='m_bet-button__odds-text').text.strip()
                    bets.append((name, odds))
                break
    else:
        print("Accordion item is not expanded.")


    driver.quit()
    return bets,game_title
    

In [77]:
all_games = []
with open('list_urls.txt', 'r') as file:
    all_games = [line.strip() for line in file.readlines()]

In [78]:
all_bets_data = []

for game_url,i in all_games, len(all_games) :
    print(f"crawling game {i+1}")
    bets = []
    game_title = ""
    for i in range(5):
        try: 
            bets, game_title = get_game(game_url)
            if bets:
                break
        except Exception as e:
            print(f"Attempt {i+1} failed for game {game_url}: {e}")
    if bets:
        for name, odds in bets:
            all_bets_data.append([game_title, name, float(odds)])



Attempt 1 failed for game https://www.swisslos.ch/de/sporttip/sportwetten/wetten/sport/70/group/4/specialGroup/fu_em/competition/2631/event/497257: 'NoneType' object has no attribute 'text'
Attempt 2 failed for game https://www.swisslos.ch/de/sporttip/sportwetten/wetten/sport/70/group/4/specialGroup/fu_em/competition/2631/event/497257: Message: element click intercepted: Element <span _ngcontent-sfg-c138="" sgelabel="" isbig="true" class="a_label a_label--big">...</span> is not clickable at point (232, 578). Other element would receive the click: <div class="footer__flyout___wrapper">...</div>
  (Session info: chrome=125.0.6422.142)
Stacktrace:
	GetHandleVerifier [0x00007FF7F0581F52+60322]
	(No symbol) [0x00007FF7F04FCEC9]
	(No symbol) [0x00007FF7F03B7EBA]
	(No symbol) [0x00007FF7F040F32E]
	(No symbol) [0x00007FF7F040CCF2]
	(No symbol) [0x00007FF7F040A18B]
	(No symbol) [0x00007FF7F0409356]
	(No symbol) [0x00007FF7F03FB491]
	(No symbol) [0x00007FF7F042C21A]
	(No symbol) [0x00007FF7F03FA

In [75]:
df = pd.DataFrame(all_bets_data, columns=["Game Title", "Result", "Odds"])
# Sort the DataFrame by game title and odds
df_sorted = df.sort_values(by=["Game Title", "Odds"])

# Keep only the 3 lowest odds for each game title
df_filtered = df_sorted.groupby("Game Title").head(3)

# Display the filtered DataFrame
display(df_filtered)



,Game Title,Result,Odds
1,Deutschland – Schottland,2:0,5.11
0,Deutschland – Schottland,1:0,5.84
3,Deutschland – Schottland,3:0,6.54
